In [97]:
from pymongo import MongoClient
import pandas as pd
import re
import getpass
import json
import os
import requests
from dotenv import load_dotenv
load_dotenv()
import numpy as np
import functions as f

- Connecting with 1M companies (the mongodb imported with only companies that have raised more tha 1M$

In [3]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [4]:
db, coll = connectCollection('companies','1MCompanies')

In [98]:
company_locations = list(coll.find({},{"Latitude":1,"Longitude":1}))
len(company_locations)

4499

- Using foursquare API to find the nearest starbucks in a radius of 500m for each company office

In [118]:
def starbucksRequestAuthorized(params):
    authToken1 = os.getenv("CLIENT_ID")
    authToken2 = os.getenv("CLIENT_SECRET")
    if not authToken1 or not authToken2:
        raise ValueError("NECESITAS UN TOKEN")
    else:
        params["client_id"] = authToken1
        params["client_secret"] = authToken2
        url = 'https://api.foursquare.com/v2/venues/explore'
        res = requests.get(url, params=params)

    return res

In [119]:
def apistarbucks(long,lat):
    location = {"v":"20180323",
               "ll":f'{lat},{long}',
               "query":"Starbucks",
               "limit":1,
               "radius":500}
    data = json.loads(starbucksRequestAuthorized(location).text)
    return data

In [126]:
all_starbucks=[]
for company in company_locations:
    starbucks=dict()
    comp_lat=company['Latitude']
    comp_long=company['Longitude']
    dat=apistarbucks(comp_long,comp_lat)
    resp=dat['response']
    if resp!={} and resp['totalResults']>0:
        lt=dat['response']['groups'][0]['items'][0]['venue']['location']['lat']
        lg=dat['response']['groups'][0]['items'][0]['venue']['location']['lng']
        starbucks['refcomp']=str(company['_id'])
        starbucks['latitude']=lt
        starbucks['longitude']=lg
        starbucks['geoloc_stb']=f.getLocation(lg,lt)
        all_starbucks.append(starbucks)

In [153]:
starbucks_df=pd.DataFrame(all_starbucks)

In [154]:
starbucks_df.to_json('./output/OfficessWithStarbucks.json', orient="records")

OverflowError: Unsupported UTF-8 sequence length when encoding string

In [161]:
# Felipe es genial
with open("starbucks.json", "w+") as file:
    file.write(str(all_starbucks))